In [190]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [191]:
#This notebook requires the following modules
import os
import numpy
import csv

In [192]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [193]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [194]:
events.count('2756') #catch trials

4

In [195]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [196]:
#uberlist

In [197]:
'2810' in uberlist[74][2]

False

In [198]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [199]:
#RT

In [200]:
#The following provides indexed lists for each of the individual event codes of interest 

In [201]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [202]:
#float(times[74])-float(times[70])

In [203]:
#Saccade

In [204]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [205]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [206]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [207]:
#print(TimeTrials[0])

In [208]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [209]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [210]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [211]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [212]:
#RT

In [213]:
checksum = sum(error)+sum(abort)+sum(catch)

In [214]:
checksum

161

In [215]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0 180
2 1 0 0 0 0 0 45
3 0 1 0 1 0 194.02770000000055 45
4 0 1 0 1 0 166.01090000000113 135
5 0 1 0 1 0 205.98790000000008 180
6 1 0 0 0 0 0 0
7 1 0 0 0 0 0 135
8 0 1 0 1 0 185.95840000000135 45
9 1 0 0 0 0 0 45
10 0 1 0 1 0 175.96410000000105 315
11 0 1 0 1 0 211.96800000000076 135
12 1 0 0 0 0 0 225
13 0 0 0 0 1 0 315
14 0 1 0 1 0 178.01209999999992 0
15 0 1 0 1 0 179.97829999999885 90
16 0 1 0 1 0 198.0007000000005 180
17 0 1 0 1 0 217.9893000000011 45
18 1 0 0 0 0 0 45
19 0 1 0 1 0 183.99239999999918 0
20 1 0 0 0 0 0 315
21 1 0 0 0 0 0 45
22 0 1 0 1 0 227.98340000000462 225
23 1 0 0 0 0 0 90
24 0 1 0 1 0 170.06609999999637 225
25 1 0 0 0 0 0 45
26 0 1 0 1 0 161.99679999999353 225
27 0 1 0 1 0 189.97269999999844 45
28 0 1 0 1 0 156.01680000000488 0
29 0 1 0 1 0 188.0063999999984 0
30 0 1 0 1 0 157.98259999999573 270
31 0 0 1 0 0 169.94310000000405 45
32 0 1 0 1 0 205.98769999999786 90
33 0 1 0 1 0 224.01019999999698 225
34 0 1 0 1 0 193.98659999999654 180
35 0 1 0 1 0 18

256 0 1 0 1 0 273.98159999994095 135
257 1 0 0 0 0 0 135
258 0 1 0 1 0 229.99049999995623 0
259 1 0 0 0 0 0 0
260 0 1 0 1 0 179.97830000007525 180
261 0 1 0 1 0 245.9647999999579 0
262 1 0 0 0 0 0 45
263 1 0 0 0 0 0 0
264 0 1 0 1 0 189.9727000000421 90
265 1 0 0 0 0 0 135
266 1 0 0 0 0 0 225
267 0 1 0 1 0 240.02579999994487 225
268 1 0 0 0 0 0 45
269 0 1 0 1 0 311.9513999999035 0
270 0 1 0 1 0 251.98600000003353 45
271 0 1 0 1 0 224.01029999996535 45
272 1 0 0 0 0 0 180
273 1 0 0 0 0 0 135
274 0 1 0 1 0 273.9812999999849 180
275 0 1 0 1 0 283.9756000000052 180
276 0 1 0 1 0 246.00580000004265 135
277 0 1 0 1 0 255.99980000010692 225
278 0 1 0 1 0 225.9762000000337 135
279 0 1 0 1 0 222.00329999998212 270
280 0 1 0 1 0 223.96920000005048 45
281 0 1 0 1 0 210.00189999991562 135
282 1 0 0 0 0 0 180
283 1 0 0 0 0 0 270
284 1 0 0 0 0 0 45
285 0 1 0 1 0 209.96100000001024 90
286 1 0 0 0 0 0 315
287 1 0 0 0 0 0 180
288 1 0 0 0 0 0 180
289 1 0 0 0 0 0 0
290 0 1 0 1 0 215.9819000000134 180
291 

604 1 0 0 0 0 0 225
605 1 0 0 0 0 0 90
606 0 0 0 1 0 347.9961999999359 90
607 0 1 0 1 0 277.99560000002384 45
608 0 1 0 1 0 223.9285000001546 225
609 0 1 0 1 0 207.95390000008047 135
610 0 1 0 1 0 229.99040000000969 315
611 0 1 0 1 0 203.98059999989346 315
612 0 1 0 1 0 241.99150000000373 0
613 0 1 0 1 0 193.98640000005253 315
614 1 0 0 0 0 0 90
615 0 1 0 1 0 275.9883999999147 0
616 0 1 0 1 0 195.9936999999918 225
617 0 1 0 1 0 215.98219999996945 225
618 0 1 0 1 0 249.97879999992438 135
619 0 1 0 1 0 225.9761999999173 180
620 0 1 0 1 0 234.00450000003912 315
621 0 1 0 1 0 263.98729999992065 180
622 0 1 0 1 0 246.00579999992624 225
623 0 1 0 1 0 209.96099999989383 0
624 0 1 0 1 0 193.98660000017844 315
625 0 1 0 1 0 201.97399999992922 180
626 0 1 0 1 0 226.01730000018142 90
627 0 1 0 1 0 231.95650000008754 45
628 1 0 0 0 0 0 315
629 1 0 0 0 0 0 0
630 0 1 0 1 0 239.9845000000205 180
631 0 1 0 1 0 215.98219999996945 135
632 0 1 0 1 0 183.99210000014864 180
633 0 1 0 1 0 181.90339999995194

In [216]:
sum(catch)

4